In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np

from util import read_passages
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, confusion_matrix
from mlxtend.evaluate import mcnemar

In [2]:
testfilename = "lucky_test.txt"
test_seqs, test_labels = read_passages(testfilename, is_labeled=True)

In [3]:
def linearize(labels):
    linearized = []
    for paper in labels:
        for label in paper:
            linearized.append(label)
    return linearized

In [4]:
def read_label(pred_file):
    all_labels = []
    labels = []
    with open(pred_file,"r") as f:
        for line in f:
            label = line.strip()
            if len(label)>0:
                labels.append(label)
            else:
                all_labels.append(labels)
                labels = []
        all_labels.append(labels)
    return all_labels

In [6]:
bert_predictions = read_label("predictions/lucky_testatt=True_cont=LSTM_clause_lstm=False_bi=True_crf=True.out")
scidt_predictions = read_label("scidt_test.out")

In [7]:
linearized_gold = linearize(test_labels)
linearized_predictions_bert = linearize(bert_predictions)
linearized_predictions_scidt = linearize(scidt_predictions)

In [8]:
f1_score(linearized_gold,linearized_predictions_bert,average="weighted")

0.8433908497588596

In [9]:
yesyes = 0
yesno = 0
noyes = 0
nono = 0
for bert_label, scidt_label, gold_label in zip(linearized_predictions_bert, linearized_predictions_scidt, linearized_gold):
    if bert_label==gold_label and scidt_label==gold_label:
        yesyes += 1
    elif bert_label==gold_label and scidt_label!=gold_label:
        yesno += 1
    elif bert_label!=gold_label and scidt_label==gold_label:
        noyes += 1
    elif bert_label!=gold_label and scidt_label!=gold_label:
        nono += 1
table = [[yesyes,yesno],[noyes,nono]]

In [10]:
stat,p = mcnemar(ary = np.array(table), corrected=True)

In [11]:
p

0.0019777271411328635